# Cross-Match KOA and Megafile

This code compares the HGCA megafile (containing all HGCA stars) with the current KOA database (the publically available Keck database). 

The code will locate HGCA stars observed by Keck and output a file containing the parameters for each star. This can then be fed into a spreadsheet. 

In [1]:
# Importing necessary modules
import numpy as np 
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
import csv 
import sys
import scipy
from astropy.time import Time

In [2]:
# Reading in the HGCA megafile
megafile = pd.read_csv('/Users/Jess/HGCA_survey_paper/megafile.csv')
print('There are ' + str(len(megafile)) + ' HCGA stars.')

There are 115663 HCGA stars.


In [3]:
# Reading in KOA file
koafile = pd.read_csv('/Users/Jess/Downloads/new_keck_stars_edited.csv')
print('There are ' + str(len(koafile)) + ' files in the KOA database.')

There are 335474 files in the KOA database.


It will take a long time to shift every megafile star by every date line in the KOA file so firstly we find the dates that appear, so we only need to shift the RA and Dec once. 

In [5]:
# Create a list of unique observation dates
myset = set(koafile['date'])
myset = list(myset)
print('The KOA list of files has ' + str(len(koafile['date'])) + ' files, which have ' + str(len(myset)) + ' unique observation dates.')

The KOA list of files has 335474 files, which have 1784 unique observation dates.


Then we need to create a catalog out of the megafile so that it can be used with the astropy module. For the observation time/epoch, we will use an average of the gaia RA and Dec epochs.

In [6]:
# Determine the average epoch time
ave_time = (megafile['epoch_ra_gaia'] + megafile['epoch_dec_gaia']) / 2

In [7]:
# Set up catalog of all the stars in the megafile
c = SkyCoord(ra = list(megafile['ra'])*u.deg, dec = list(megafile['dec'])*u.deg, 
distance = Distance(parallax = list(megafile['gaia_parallax']) * u.mas), 
pm_ra_cosdec = list(megafile['pmra_gaia']) * u.mas/u.yr, 
pm_dec = list(megafile['pmdec_gaia']) * u.mas/u.yr, 
obstime = Time(ave_time, format='decimalyear'))
print(len(c))

115663


We also need the KOA file list as a catalog. 

This file has some RA and Dec values in it which are NaN and cannot be accepted. So the first step is to clean up the catalog to not include these files. 

In [8]:
# Set up a catalog of...
catalog = SkyCoord(ra=koafile['ra']*u.degree, dec=koafile['dec']*u.degree)
clean = (~np.isnan(catalog.ra) & ~np.isnan(catalog.dec))
print('The initial file has ' + str(len(catalog)) + ' lines,') 
print('After being cleaned it has ' + str(clean.sum()) + ' lines,')
print('Meaning that ' + str(len(catalog)-clean.sum()) + ' files have been removed.')
catalog_clean = catalog[clean]

The initial file has 335474 lines,
After being cleaned it has 335141 lines,
Meaning that 333 files have been removed.


/opt/miniconda3/lib/python3.7/site-packages/astropy/coordinates/angles.py:522: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/opt/miniconda3/lib/python3.7/site-packages/astropy/coordinates/angles.py:522: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):


This loop first shifts the ra and dec of every star in the megafile to a new position, given the date in the KOA file. 

It then compares all of these new positions to positions in the KOA file to see if it can find any matches within 10 arcseconds of the central coordinate for the KOA file. 

NB: To speed up the process, we use the consolidated list of dates so that each date is only used to shift the megafile once.

In [9]:
for j in range(len(myset)):
    print('Running for KOA date No# ' + str(j) + ': ' + str(myset[j])) 
    sc = c.apply_space_motion(new_obstime=Time(myset[j]))
    idxsc, idxcatalog, d2d, d3d = catalog_clean.search_around_sky(sc, 20*u.arcsecond)

Running for KOA date No# 0: 2013-07-23
Running for KOA date No# 1: 2011-12-17
Running for KOA date No# 2: 2015-08-28
Running for KOA date No# 3: 2009-06-27
Running for KOA date No# 4: 2017-04-14
Running for KOA date No# 5: 2007-09-23
Running for KOA date No# 6: 2016-09-18
Running for KOA date No# 7: 2012-07-04
Running for KOA date No# 8: 2002-02-28
Running for KOA date No# 9: 2003-10-05
Running for KOA date No# 10: 2010-11-12
Running for KOA date No# 11: 2007-12-02
Running for KOA date No# 12: 2010-09-05
Running for KOA date No# 13: 2012-01-01
Running for KOA date No# 14: 2004-01-13
Running for KOA date No# 15: 2018-05-24
Running for KOA date No# 16: 2016-02-27
Running for KOA date No# 17: 2012-01-12
Running for KOA date No# 18: 2012-12-23
Running for KOA date No# 19: 2004-01-12
Running for KOA date No# 20: 2007-04-23
Running for KOA date No# 21: 2015-07-22
Running for KOA date No# 22: 2007-03-23
Running for KOA date No# 23: 2007-09-06
Running for KOA date No# 24: 2017-05-27
Running fo

Running for KOA date No# 203: 2017-07-20
Running for KOA date No# 204: 2012-03-28
Running for KOA date No# 205: 2013-03-02
Running for KOA date No# 206: 2009-12-08
Running for KOA date No# 207: 2018-01-18
Running for KOA date No# 208: 2014-05-21
Running for KOA date No# 209: 2017-01-09
Running for KOA date No# 210: 2007-05-14
Running for KOA date No# 211: 2007-08-07
Running for KOA date No# 212: 2017-09-06
Running for KOA date No# 213: 2016-04-17
Running for KOA date No# 214: 2013-10-12
Running for KOA date No# 215: 2013-04-26
Running for KOA date No# 216: 2017-04-13
Running for KOA date No# 217: 2011-12-27
Running for KOA date No# 218: 2013-10-16
Running for KOA date No# 219: 2002-10-27
Running for KOA date No# 220: 2014-09-03
Running for KOA date No# 221: 2006-06-10
Running for KOA date No# 222: 2017-06-12
Running for KOA date No# 223: 2010-05-02
Running for KOA date No# 224: 2017-07-13
Running for KOA date No# 225: 2015-04-11
Running for KOA date No# 226: 2012-05-23
Running for KOA 

Running for KOA date No# 403: 2013-07-18
Running for KOA date No# 404: 2012-10-04
Running for KOA date No# 405: 2017-06-03
Running for KOA date No# 406: 2010-09-26
Running for KOA date No# 407: 2002-10-28
Running for KOA date No# 408: 2016-03-24
Running for KOA date No# 409: 2004-05-28
Running for KOA date No# 410: 2012-07-10
Running for KOA date No# 411: 2008-08-10
Running for KOA date No# 412: 2018-06-06
Running for KOA date No# 413: 2007-07-29
Running for KOA date No# 414: 2009-12-11
Running for KOA date No# 415: 2009-09-08
Running for KOA date No# 416: 2005-11-09
Running for KOA date No# 417: 2003-09-19
Running for KOA date No# 418: 2016-11-14
Running for KOA date No# 419: 2008-01-13
Running for KOA date No# 420: 2010-03-30
Running for KOA date No# 421: 2015-02-06
Running for KOA date No# 422: 2017-06-10
Running for KOA date No# 423: 2002-04-25
Running for KOA date No# 424: 2012-12-29
Running for KOA date No# 425: 2005-10-20
Running for KOA date No# 426: 2018-02-01
Running for KOA 

Running for KOA date No# 603: 2009-07-19
Running for KOA date No# 604: 2013-04-29
Running for KOA date No# 605: 2003-07-29
Running for KOA date No# 606: 2001-12-01
Running for KOA date No# 607: 2015-08-30
Running for KOA date No# 608: 2016-06-16
Running for KOA date No# 609: 2011-04-21
Running for KOA date No# 610: 2014-04-10
Running for KOA date No# 611: 2003-04-15
Running for KOA date No# 612: 2017-12-05
Running for KOA date No# 613: 2005-10-28
Running for KOA date No# 614: 2007-08-20
Running for KOA date No# 615: 2017-08-25
Running for KOA date No# 616: 2004-08-10
Running for KOA date No# 617: 2017-04-09
Running for KOA date No# 618: 2018-01-30
Running for KOA date No# 619: 2013-10-14
Running for KOA date No# 620: 2012-02-02
Running for KOA date No# 621: 2015-06-15
Running for KOA date No# 622: 2013-07-27
Running for KOA date No# 623: 2002-09-21
Running for KOA date No# 624: 2008-05-11
Running for KOA date No# 625: 2002-08-21
Running for KOA date No# 626: 2014-11-02
Running for KOA 

Running for KOA date No# 803: 2011-09-22
Running for KOA date No# 804: 2015-01-25
Running for KOA date No# 805: 2016-10-19
Running for KOA date No# 806: 2004-07-08
Running for KOA date No# 807: 2011-06-18
Running for KOA date No# 808: 2003-12-27
Running for KOA date No# 809: 2012-08-11
Running for KOA date No# 810: 2016-01-25
Running for KOA date No# 811: 2011-08-12
Running for KOA date No# 812: 2012-05-15
Running for KOA date No# 813: 2012-10-26
Running for KOA date No# 814: 2014-08-07
Running for KOA date No# 815: 2005-04-23
Running for KOA date No# 816: 2012-07-06
Running for KOA date No# 817: 2006-07-18
Running for KOA date No# 818: 2011-11-13
Running for KOA date No# 819: 2006-07-29
Running for KOA date No# 820: 2010-06-18
Running for KOA date No# 821: 2018-05-01
Running for KOA date No# 822: 2005-05-01
Running for KOA date No# 823: 2019-07-16
Running for KOA date No# 824: 2018-08-22
Running for KOA date No# 825: 2013-05-30
Running for KOA date No# 826: 2017-09-22
Running for KOA 

Running for KOA date No# 1003: 2009-09-05
Running for KOA date No# 1004: 2016-08-19
Running for KOA date No# 1005: 2018-01-10
Running for KOA date No# 1006: 2012-06-04
Running for KOA date No# 1007: 2019-06-10
Running for KOA date No# 1008: 2016-05-27
Running for KOA date No# 1009: 2012-01-14
Running for KOA date No# 1010: 2017-09-02
Running for KOA date No# 1011: 2008-09-19
Running for KOA date No# 1012: 2016-10-14
Running for KOA date No# 1013: 2016-03-21
Running for KOA date No# 1014: 2011-04-13
Running for KOA date No# 1015: 2010-11-29
Running for KOA date No# 1016: 2017-08-19
Running for KOA date No# 1017: 2016-11-08
Running for KOA date No# 1018: 2012-08-09
Running for KOA date No# 1019: 2006-11-26
Running for KOA date No# 1020: 2001-12-22
Running for KOA date No# 1021: 2015-07-31
Running for KOA date No# 1022: 2015-11-23
Running for KOA date No# 1023: 2010-07-28
Running for KOA date No# 1024: 2006-08-10
Running for KOA date No# 1025: 2011-10-12
Running for KOA date No# 1026: 201

Running for KOA date No# 1199: 2004-03-11
Running for KOA date No# 1200: 2009-11-02
Running for KOA date No# 1201: 2010-04-04
Running for KOA date No# 1202: 2013-09-21
Running for KOA date No# 1203: 2017-12-08
Running for KOA date No# 1204: 2006-02-07
Running for KOA date No# 1205: 2009-05-31
Running for KOA date No# 1206: 2012-07-08
Running for KOA date No# 1207: 2014-09-10
Running for KOA date No# 1208: 2011-11-12
Running for KOA date No# 1209: 2005-09-12
Running for KOA date No# 1210: 2015-08-08
Running for KOA date No# 1211: 2011-01-06
Running for KOA date No# 1212: 2014-10-11
Running for KOA date No# 1213: 2014-12-10
Running for KOA date No# 1214: 2016-08-06
Running for KOA date No# 1215: 2015-05-05
Running for KOA date No# 1216: 2017-12-04
Running for KOA date No# 1217: 2009-09-10
Running for KOA date No# 1218: 2006-05-21
Running for KOA date No# 1219: 2003-05-17
Running for KOA date No# 1220: 2015-04-30
Running for KOA date No# 1221: 2005-02-16
Running for KOA date No# 1222: 201

Running for KOA date No# 1395: 2014-10-12
Running for KOA date No# 1396: 2015-07-12
Running for KOA date No# 1397: 2006-03-21
Running for KOA date No# 1398: 2017-05-05
Running for KOA date No# 1399: 2016-02-05
Running for KOA date No# 1400: 2007-09-19
Running for KOA date No# 1401: 2009-05-09
Running for KOA date No# 1402: 2017-11-16
Running for KOA date No# 1403: 2011-06-27
Running for KOA date No# 1404: 2007-07-12
Running for KOA date No# 1405: 2004-06-15
Running for KOA date No# 1406: 2017-01-24
Running for KOA date No# 1407: 2007-04-05
Running for KOA date No# 1408: 2017-03-14
Running for KOA date No# 1409: 2003-07-06
Running for KOA date No# 1410: 2011-11-10
Running for KOA date No# 1411: 2002-03-03
Running for KOA date No# 1412: 2013-03-25
Running for KOA date No# 1413: 2003-06-07
Running for KOA date No# 1414: 2004-09-01
Running for KOA date No# 1415: 2008-03-31
Running for KOA date No# 1416: 2010-08-17
Running for KOA date No# 1417: 2006-05-05
Running for KOA date No# 1418: 201

Running for KOA date No# 1591: 2013-06-27
Running for KOA date No# 1592: 2007-03-26
Running for KOA date No# 1593: 2012-07-09
Running for KOA date No# 1594: 2017-05-28
Running for KOA date No# 1595: 2011-03-25
Running for KOA date No# 1596: 2015-04-05
Running for KOA date No# 1597: 2017-09-11
Running for KOA date No# 1598: 2002-10-21
Running for KOA date No# 1599: 2004-05-25
Running for KOA date No# 1600: 2015-03-03
Running for KOA date No# 1601: 2016-11-24
Running for KOA date No# 1602: 2016-01-03
Running for KOA date No# 1603: 2013-08-18
Running for KOA date No# 1604: 2014-07-29
Running for KOA date No# 1605: 2006-07-17
Running for KOA date No# 1606: 2013-07-03
Running for KOA date No# 1607: 2010-08-23
Running for KOA date No# 1608: 2002-06-17
Running for KOA date No# 1609: 2002-09-27
Running for KOA date No# 1610: 2019-05-26
Running for KOA date No# 1611: 2015-04-10
Running for KOA date No# 1612: 2013-01-19
Running for KOA date No# 1613: 2011-10-21
Running for KOA date No# 1614: 200